In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, 

SyntaxError: trailing comma not allowed without surrounding parentheses (<ipython-input-6-4f7df032f433>, line 7)

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')

In [29]:
df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [93]:
df.columns

Index(['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke'], dtype='object')

In [38]:
df['stroke'].value_counts()

0    4861
1     249
Name: stroke, dtype: int64

In [8]:
cat_features = ['gender','ever_married','work_type', 'Residence_type', 'smoking_status']

gender, ever_married, work_type, Residence_type, smoking_status - Categorical Encoding   
bmi - KNN Imputation

In [5]:
X = df.drop(columns=['id','stroke'])
y = df['stroke']

In [48]:
X

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked
...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=420)

In [6]:
X.shape , y.shape

((5110, 10), (5110,))

In [75]:
preprocess_pipe = ColumnTransformer(transformers= [
    ('one_hot_encode', OneHotEncoder(drop= 'first') , [0,4,5,6,9]),
    ('knn_impute', KNNImputer(n_neighbors= 10, weights= 'distance'), [8]),
] , remainder= 'passthrough')

In [76]:
preprocess_pipe

ColumnTransformer(remainder='passthrough',
                  transformers=[('one_hot_encode', OneHotEncoder(drop='first'),
                                 [0, 4, 5, 6, 9]),
                                ('knn_impute',
                                 KNNImputer(n_neighbors=10, weights='distance'),
                                 [8])])

In [77]:
X_new = preprocess_pipe.fit_transform(X)

In [78]:
X_new[0]

array([  1.  ,   0.  ,   1.  ,   0.  ,   1.  ,   0.  ,   0.  ,   1.  ,
         1.  ,   0.  ,   0.  ,  36.6 ,  67.  ,   0.  ,   1.  , 228.69])

In [79]:
X_new.shape

(5110, 16)

In [80]:
model_pipeline = Pipeline(steps= [('preprocess', preprocess_pipe), ('random_forest', RandomForestClassifier(n_estimators= 500 , n_jobs= -1, max_depth=2, criterion= 'entropy'))])

In [81]:
model_pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot_encode',
                                                  OneHotEncoder(drop='first'),
                                                  [0, 4, 5, 6, 9]),
                                                 ('knn_impute',
                                                  KNNImputer(n_neighbors=10,
                                                             weights='distance'),
                                                  [8])])),
                ('random_forest',
                 RandomForestClassifier(criterion='entropy', max_depth=2,
                                        n_estimators=500, n_jobs=-1))])

In [82]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot_encode',
                                                  OneHotEncoder(drop='first'),
                                                  [0, 4, 5, 6, 9]),
                                                 ('knn_impute',
                                                  KNNImputer(n_neighbors=10,
                                                             weights='distance'),
                                                  [8])])),
                ('random_forest',
                 RandomForestClassifier(criterion='entropy', max_depth=2,
                                        n_estimators=500, n_jobs=-1))])

In [83]:
y_pred_pipe = model_pipeline.predict(X_test)

In [84]:
accuracy_score(y_pred= y_pred_pipe , y_true = y_test)

0.9510763209393346

In [85]:
from joblib import dump

In [86]:
dump(model_pipeline, 'model_pipeline.joblib')

['model_pipeline.joblib']

In [87]:
df.sample(10).drop(columns=['stroke','id']).to_csv("sample.csv", index = False)

In [7]:
sample_test = pd.read_csv('sample.csv')

In [10]:
sample_test.iloc[0,:].to_json()

'{"gender":"Female","age":40.0,"hypertension":0,"heart_disease":0,"ever_married":"Yes","work_type":"Private","Residence_type":"Rural","avg_glucose_level":74.51,"bmi":36.6,"smoking_status":"never smoked"}'

In [2]:
sample_test

NameError: name 'sample_test' is not defined

In [90]:
sample_test.iloc[0,:].tolist()

['Female', 40.0, 0, 0, 'Yes', 'Private', 'Rural', 74.51, 36.6, 'never smoked']

In [91]:
test_pred = model_pipeline.predict([['Female', 40.0, 0, 0, 'Yes', 'Private', 'Rural', 74.51, 36.6, 'never smoked']])

In [94]:
test_pred[0]

0

In [1]:
import requests

url = 'http://127.0.0.1:8000/predict/'
data = {
  "gender": "Male",
  "age": 20,
  "hypertension": 0,
  "heart_disease": 0,
  "ever_married": "Yes",
  "work_type": "Private",
  "Residence_type": "Rural",
  "avg_glucose_level": 74.51,
  "bmi": 36.6,
  "smoking_status": "never smoked"
}

resp = requests.post(url, json=data)
print(resp.json())

{'prediction': 'No Stroke'}
